In [1]:
import os
import torch
import logging

import sys
import random

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib import cm
import matplotlib.animation as anim
import matplotlib.patches as mpatches
import matplotlib.gridspec as gridspec

import nibabel as nib
import torch.nn as nn
import torch.optim as optim
import gc
import torch

import torch.nn.functional as F
import os
import numpy as np
import nibabel as nib
import torch
from torch.utils.data import Dataset

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix
from tqdm import tqdm

from torch.utils.data import Dataset, DataLoader, random_split, ConcatDataset
from torch.utils.tensorboard import SummaryWriter
from torch.cuda.amp import autocast, GradScaler
from sklearn.metrics import confusion_matrix
import kagglehub
from huggingface_hub import HfApi, create_repo
import os
from datetime import datetime
from kaggle_secrets import UserSecretsClient
import cv2

2025-07-01 17:32:50.320460: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751391170.724829      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751391170.845427      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Model creation

In [18]:
class ImprovedUNet3D(nn.Module):
    def __init__(self, in_channels=4, out_channels=4, base_filters=16):
        super(ImprovedUNet3D, self).__init__()

        self.enc1 = self._make_layer(in_channels, base_filters)
        self.pool1 = nn.MaxPool3d(kernel_size=2, stride=2)

        self.enc2 = self._make_layer(base_filters, base_filters*2)
        self.pool2 = nn.MaxPool3d(kernel_size=2, stride=2)

        self.enc3 = self._make_layer(base_filters*2, base_filters*4)

        self.upconv2 = nn.ConvTranspose3d(base_filters*4, base_filters*2, kernel_size=2, stride=2)
        self.dec2 = self._make_layer(base_filters*4, base_filters*2)

        self.upconv1 = nn.ConvTranspose3d(base_filters*2, base_filters, kernel_size=2, stride=2)
        self.dec1 = self._make_layer(base_filters*2, base_filters)

        self.dropout = nn.Dropout3d(0.10)
        self.final_conv = nn.Conv3d(base_filters, out_channels, kernel_size=1)

        self._initialize_weights()

    def _make_layer(self, in_channels, out_channels):
        """Create a residual block"""
        return nn.Sequential(
            nn.Conv3d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.InstanceNorm3d(out_channels),
            nn.LeakyReLU(inplace=True),
            nn.Conv3d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.InstanceNorm3d(out_channels),
            nn.LeakyReLU(inplace=True)
        )

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv3d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.InstanceNorm3d):
                if m.weight is not None:
                    nn.init.constant_(m.weight, 1)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

    def forward(self, x):
        enc1_out = self.enc1(x)
        p1 = self.pool1(enc1_out)

        enc2_out = self.enc2(p1)
        p2 = self.pool2(enc2_out)

        enc3_out = self.enc3(p2)

        up2 = self.upconv2(enc3_out)

        diffY = enc2_out.size()[2] - up2.size()[2]
        diffX = enc2_out.size()[3] - up2.size()[3]
        diffZ = enc2_out.size()[4] - up2.size()[4]

        up2 = F.pad(up2, [
            diffZ // 2, diffZ - diffZ // 2,
            diffX // 2, diffX - diffX // 2,
            diffY // 2, diffY - diffY // 2
        ])
        concat2 = torch.cat([up2, enc2_out], dim=1)
        dec2_out = self.dec2(concat2)

        up1 = self.upconv1(dec2_out)

        diffY = enc1_out.size()[2] - up1.size()[2]
        diffX = enc1_out.size()[3] - up1.size()[3]
        diffZ = enc1_out.size()[4] - up1.size()[4]

        up1 = F.pad(up1, [
            diffZ // 2, diffZ - diffZ // 2,
            diffX // 2, diffX - diffX // 2,
            diffY // 2, diffY - diffY // 2
        ])
        concat1 = torch.cat([up1, enc1_out], dim=1)
        dec1_out = self.dec1(concat1)

        x = self.dropout(dec1_out)

        out = self.final_conv(x)

        return out

model = ImprovedUNet3D(base_filters=16)
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params:,}")
size_in_bytes = total_params * 4
size_in_mb = size_in_bytes / (1024 ** 2)
print(f"Model size (inference): {size_in_mb:.2f} MB")


Total parameters: 340,596
Model size (inference): 1.30 MB


## Dataset processing

In [4]:
class BrainTumorDataset(Dataset):
    def __init__(self, patient_list, data_dir, is_train=True, transform=None, slice_range=(60, 100)):
        self.data_dir = data_dir              # Path to directory containing patient subfolders
        self.is_train = is_train              # Flag indicating training or inference phase
        self.transform = transform            # Optional data augmentations or preprocessing
        # List of MRI modalities
        self.modalities = ["flair", "t1", "t1ce", "t2"]

        # Filter out any CSV files and exclude problematic patient 355
        self.patients = [p for p in patient_list if not p.endswith(".csv") and "355" not in p]

        # Range of axial slices to load (start_slice, end_slice)
        self.slice_range = slice_range       

    def load_nifti(self, path):
        # Load NIfTI file from disk using nibabel and return as numpy float32 array
        nifti_img = nib.load(path)
        return np.array(nifti_img.get_fdata(), dtype=np.float32)

    def __len__(self):
        return len(self.patients)

    def __getitem__(self, idx):
        patient_id = self.patients[idx]

        # Safeguard: skip patient 355 if encountered
        if patient_id == "BraTS20_Training_355":
            return None
        # Construct path to this patient's data folder
        patient_path = os.path.join(self.data_dir, patient_id)
        # Load and preprocess each MRI modality
        images = []
        for modality in self.modalities:
            img_path = os.path.join(patient_path, f"{patient_id}_{modality}.nii")
            img = self.load_nifti(img_path)
            # Crop to the specified slice range along the axial (third) dimension
            start_slice, end_slice = self.slice_range
            img = img[:, :, start_slice:end_slice]
            # Apply z-score normalization for stability; avoid division by zero
            if np.std(img) > 0:
                img = (img - np.mean(img)) / np.std(img)
            else:
                img = np.zeros_like(img)

            images.append(img)
        # Stack the four modalities into a single tensor of shape (4, H, W, D)
        images = np.stack(images, axis=0)
        # Load segmentation mask if in training mode
        if self.is_train:
            # Possible filenames for segmentation masks
            seg_paths = [
                os.path.join(patient_path, f"{patient_id}_seg.nii"),
                os.path.join(patient_path, f"{patient_id}_Segm.nii"),
                os.path.join(patient_path, f"{patient_id}_Segmentation.nii")
            ]
            # Select the first existing segmentation file
            seg_path = next((path for path in seg_paths if os.path.exists(path)), None)

            if seg_path is None:
                # Issue warning and create empty mask if not found
                print(f"Warning: No segmentation file found for patient {patient_id}.")
                mask = np.zeros_like(images[0], dtype=np.int64)
            else:
                # Load and crop mask to the same slice range
                mask = self.load_nifti(seg_path)
                mask = mask[:, :, self.slice_range[0]:self.slice_range[1]]
                # Reassign label '4' (if present) to 3 for enhancing tumor
                mask[mask == 4] = 3
        else:
            mask = None  # No mask needed for inference
        # Apply optional transformations (e.g., augmentation) before tensor conversion
        if self.transform is not None:
            if self.is_train:
                # When training, transform both image and mask
                transformed = self.transform(image=images, mask=mask)
                images, mask = transformed['image'], transformed['mask']
            else:
                # During inference, only transform the image
                transformed = self.transform(image=images)
                images = transformed['image']
        # Convert numpy arrays to PyTorch tensors
        if not isinstance(images, torch.Tensor):
            images = torch.tensor(images, dtype=torch.float32)
        if self.is_train and mask is not None:
            # Convert mask to long tensor for use in loss functions
            if not isinstance(mask, torch.Tensor):
                mask = torch.tensor(mask, dtype=torch.long)
            else:
                mask = mask.to(dtype=torch.long)

        # Return (image, mask) tuple for training or image only for inference
        return (images, mask) if self.is_train else images

### Download the dataset

In [5]:
path = kagglehub.dataset_download("awsaf49/brats20-dataset-training-validation")
data_dir = path + "/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData"
train_dir = path + "/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData"
val_dir = path +"/BraTS2020_ValidationData/MICCAI_BraTS2020_ValidationData"

### visualization of the dataset

In [6]:
def visualize_samples_with_colors(data_dir, BraTS20_Training_00n):
    flair = data_dir+ BraTS20_Training_00n + BraTS20_Training_00n +'_flair.nii'
    mask = data_dir+ BraTS20_Training_00n + BraTS20_Training_00n +'_seg.nii'
    t1 = data_dir+ BraTS20_Training_00n + BraTS20_Training_00n +'_t1.nii'
    t2 = data_dir+ BraTS20_Training_00n + BraTS20_Training_00n +'_t2.nii'
    t1ce = data_dir+ BraTS20_Training_00n + BraTS20_Training_00n +'_t1ce.nii'

    flair_img = nib.load(flair)
    flair_img = np.asanyarray(flair_img.dataobj)
    flair_img = np.rot90(flair_img)

    sample_mask = nib.load(mask)
    sample_mask = np.asanyarray(sample_mask.dataobj)
    sample_mask = np.rot90(sample_mask)

    t1_img = nib.load(t1)
    t1_img = np.asanyarray(t1_img.dataobj)
    t1_img  = np.rot90(t1_img)

    t2_img = nib.load(t2)
    t2_img = np.asanyarray(t2_img.dataobj)
    t2_img  = np.rot90(t2_img)

    t1ce_img = nib.load(t1ce)
    t1ce_img = np.asanyarray(t1ce_img.dataobj)
    t1ce_img  = np.rot90(t1ce_img)

    mask_WT = sample_mask.copy()
    mask_WT[mask_WT == 1] = 1
    mask_WT[mask_WT == 2] = 1
    mask_WT[mask_WT == 4] = 1

    mask_TC = sample_mask.copy()
    mask_TC[mask_TC == 1] = 1
    mask_TC[mask_TC == 2] = 0
    mask_TC[mask_TC == 4] = 1

    mask_ET = sample_mask.copy()
    mask_ET[mask_ET == 1] = 0
    mask_ET[mask_ET == 2] = 0
    mask_ET[mask_ET == 4] = 1

    fig = plt.figure(figsize=(20, 10))

    gs = gridspec.GridSpec(nrows=2, ncols=5, height_ratios=[1, 1.5])

    fontsize = 10
    ax0 = fig.add_subplot(gs[0, 0])
    flair = ax0.imshow(flair_img[:,:,65], cmap='bone')
    ax0.set_title("FLAIR", fontsize = fontsize, weight='bold', y=-0.2)
    fig.colorbar(flair)

    ax1 = fig.add_subplot(gs[0, 1])
    t1 = ax1.imshow(t1_img[:,:,65], cmap='bone')
    ax1.set_title("T1", fontsize = fontsize, weight='bold', y=-0.2)
    fig.colorbar(t1)
    
    ax2 = fig.add_subplot(gs[0, 2])
    t2 = ax2.imshow(t2_img[:,:,65], cmap='bone')
    ax2.set_title("T2", fontsize = fontsize, weight='bold', y=-0.2)
    fig.colorbar(t2)

    ax3 = fig.add_subplot(gs[0, 3])
    t1ce = ax3.imshow(t1ce_img[:,:,65], cmap='bone')
    ax3.set_title("T1 contrast", fontsize = fontsize, weight='bold', y=-0.2)
    fig.colorbar(t1ce)

    ax4 = fig.add_subplot(gs[1, 1:3])

    l1 = ax4.imshow(mask_WT[:,:,65], cmap='summer',)
    l2 = ax4.imshow(np.ma.masked_where(mask_TC[:,:,65]== False,  mask_TC[:,:,65]), cmap='rainbow', alpha=0.6)
    l3 = ax4.imshow(np.ma.masked_where(mask_ET[:,:,65] == False, mask_ET[:,:,65]), cmap='winter', alpha=0.6)

    ax4.set_title("", fontsize=fontsize, weight='bold', y=-0.1)

    _ = [ax.set_axis_off() for ax in [ax0,ax1,ax2,ax3, ax4]]

    colors = [im.cmap(im.norm(1)) for im in [l1,l2, l3]]
    labels = ['Non-Enhancing tumor core', 'Peritumoral Edema ', 'GD-enhancing tumor']
    patches = [ mpatches.Patch(color=colors[i], label=f"{labels[i]}") for i in range(len(labels))]
    plt.legend(handles=patches, bbox_to_anchor=(1.1, 0.65), loc=2, borderaxespad=0.4,fontsize = 'xx-large',
              title='Mask Labels', title_fontsize=18, edgecolor="black",  facecolor='#c5c6c7')

    plt.suptitle("Multimodal Scans -  Data | Manually-segmented mask - Target", fontsize=20, weight='bold')

    fig.savefig("data_sample.png", format="png",  pad_inches=0.2, transparent=False, bbox_inches='tight')
    fig.savefig("data_sample.svg", format="svg",  pad_inches=0.2, transparent=False, bbox_inches='tight')

## Dataset splitting and processing


In [7]:
train_patient_dirs = os.listdir(train_dir)
test_patient_dirs = os.listdir(val_dir)
all_train_patients = []
for p in train_patient_dirs:
    if os.path.isdir(os.path.join(train_dir, p)):
        all_train_patients.append(p)
all_train_patients.sort()

all_test_patients = []
for p in test_patient_dirs:
    if os.path.isdir(os.path.join(val_dir, p)):
        all_test_patients.append(p)
all_test_patients.sort()


slice_range = (0, 155)
slice_range = (0, 155)
train_patients, val_patients = train_test_split(all_train_patients, test_size=0.2, random_state=42)
train_dataset = BrainTumorDataset(
    patient_list=train_patients,
    data_dir=train_dir,
    slice_range=slice_range,
    transform=None
)

val_dataset = BrainTumorDataset(
    patient_list=val_patients,
    data_dir=train_dir,
    slice_range=slice_range
)

test_dataset = BrainTumorDataset(
    patient_list=all_test_patients,
    data_dir=val_dir,
    is_train=False,
    slice_range=slice_range
)
print(f"Using {len(train_dataset)} training patients and {len(val_dataset)} test patients")

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=2)

sample_img, sample_mask = next(iter(train_loader))
print(f"Train set image dimension: {sample_img.shape}")  # Should be smaller now
print(f"Train set segmentation dimension: {sample_mask.shape}")

sample_img, sample_mask = next(iter(val_loader))
print("Validation set image dimension:", sample_img.shape)
print("Validation set segmentation dimension:", sample_mask.shape)

sample_img = next(iter(test_loader))
print("Test set image dimension:", sample_img.shape)

Using 294 training patients and 74 test patients
Train set image dimension: torch.Size([2, 4, 240, 240, 155])
Train set segmentation dimension: torch.Size([2, 240, 240, 155])
Validation set image dimension: torch.Size([1, 4, 240, 240, 155])
Validation set segmentation dimension: torch.Size([1, 240, 240, 155])
Test set image dimension: torch.Size([1, 4, 240, 240, 155])


### set devices

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    gc.collect()
    for i in range(5): 
        gc.collect()
        torch.cuda.empty_cache()

    num_gpus = torch.cuda.device_count()
    print(f"Using {num_gpus} GPUs for training." if num_gpus > 1 else
          f"Using {torch.cuda.get_device_name(0)} for training.")

    # Print available and total memory
    if hasattr(torch.cuda, 'get_device_properties'):
        for i in range(num_gpus):
            prop = torch.cuda.get_device_properties(i)
            print(f"GPU {i}: {prop.name}")
            print(f"  Total memory: {prop.total_memory / 1024**3:.2f} GB")
            print(f"  Available memory: {torch.cuda.memory_reserved(i) / 1024**3:.2f} GB reserved")
            print(f"  Allocated memory: {torch.cuda.memory_allocated(i) / 1024**3:.2f} GB")
else:
    print("Using CPU for training.")

Using 2 GPUs for training.
GPU 0: Tesla T4
  Total memory: 14.74 GB
  Available memory: 0.00 GB reserved
  Allocated memory: 0.00 GB
GPU 1: Tesla T4
  Total memory: 14.74 GB
  Available memory: 0.00 GB reserved
  Allocated memory: 0.00 GB


In [9]:
def dice_coefficient_soft(pred_probs, target, num_classes=4, smooth=1e-6):
    dice_scores = []
    target_one_hot = F.one_hot(target, num_classes=num_classes).permute(0, 4, 1, 2, 3).float()
    for class_id in range(num_classes):
        pred_class = pred_probs[:, class_id, ...]
        target_class = target_one_hot[:, class_id, ...]
        intersection = (pred_class * target_class).sum(dim=[1, 2, 3]) # Sum over spatial dims
        pred_sum = pred_class.sum(dim=[1, 2, 3])
        target_sum = target_class.sum(dim=[1, 2, 3])
        dice_score_class = (2. * intersection + smooth) / (pred_sum + target_sum + smooth)
        dice_scores.append(dice_score_class.mean())
    dice_scores = torch.stack(dice_scores)
    return dice_scores.mean() 

# Modified DiceLoss using Soft Dice
class DiceLoss(nn.Module):
    def __init__(self, num_classes=4, smooth=1e-6):
        super(DiceLoss, self).__init__()
        self.num_classes = num_classes
        self.smooth = smooth

    def forward(self, y_pred_logits, y_true):
        y_pred_probs = F.softmax(y_pred_logits, dim=1)
        dice = dice_coefficient_soft(y_pred_probs, y_true, num_classes=self.num_classes, smooth=self.smooth)
        return 1 - dice

class CombinedLoss(nn.Module):
    def __init__(self, weight_dice=0.7, weight_ce=0.3, num_classes=4):
        super(CombinedLoss, self).__init__()
        self.weight_dice = weight_dice
        self.weight_ce = weight_ce
        self.dice_loss = DiceLoss(num_classes=num_classes)
        self.ce_loss = nn.CrossEntropyLoss()

    def forward(self, inputs_logits, targets):
        ce_loss_value = self.ce_loss(inputs_logits, targets)
        dice_loss_value = self.dice_loss(inputs_logits, targets)
        return self.weight_ce * ce_loss_value + self.weight_dice * dice_loss_value

def dice_coefficient_metric(pred_labels, target_labels, num_classes=4, smooth=1e-6):
    dice_scores = []
    for class_id in range(num_classes):
        pred_class = (pred_labels == class_id).float()
        target_class = (target_labels == class_id).float()
        intersection = (pred_class * target_class).sum()
        dice_score = (2. * intersection + smooth) / (pred_class.sum() + target_class.sum() + smooth)
        dice_scores.append(dice_score)
    dice_scores = torch.stack(dice_scores)
    return dice_scores.mean()

## Create the model

In [19]:
model = ImprovedUNet3D(in_channels=4, out_channels=4, base_filters=16)
model = torch.nn.DataParallel(model)
model.to(device)
# Use mixed precision for memory efficiency
use_amp = True
scaler = torch.amp.GradScaler("cuda", enabled="use_amp")
criterion = CombinedLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.0005, weight_decay=5e-3)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=30, eta_min=1e-6)

In [11]:
import torch

def train_one_epoch(model, dataloader, optimizer, criterion, device, scaler=None):
    """
    Train the model for one epoch.
    Args:
        model: PyTorch model to train.
        dataloader: DataLoader providing (inputs, targets).
        optimizer: Optimizer for parameter updates.
        criterion: Loss function to compute training loss.
        device: Torch device (cpu or cuda).
        scaler: Optional GradScaler for mixed precision.
    Returns:
        avg_loss: Average training loss over batches.
        avg_dice: Average Dice score over batches.
        avg_dice_per_class: Tensor of average Dice per class.
    """
    use_amp = scaler is not None  # Flag whether to use automatic mixed precision
    model.train()                 # Set model to training mode
    epoch_loss = 0.0              # Cumulative loss for the epoch
    dice_score = 0.0              # Cumulative Dice score
    dice_per_class = torch.zeros(4, device=device)  # Sum of per-class Dice scores
    batch_count = 0               # Number of processed batches

    for i, (inputs, targets) in enumerate(dataloader):
        # Move data to target device
        inputs = inputs.to(device)
        targets = targets.to(device)
        optimizer.zero_grad()      # Reset gradients

        if use_amp:
            # Use mixed precision context
            with torch.cuda.amp.autocast(enabled=True):
                outputs = model(inputs)             # Forward pass
                loss = criterion(outputs, targets)  # Compute loss
            # Scale loss, backpropagate, and unscale gradients
            scaler.scale(loss).backward()
            scaler.unscale_(optimizer)
            # Gradient clipping for stability
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.5)
            scaler.step(optimizer)
            scaler.update()
        else:
            outputs = model(inputs)                 # Forward pass
            loss = criterion(outputs, targets)      # Compute loss
            loss.backward()                         # Backpropagate
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.5)
            optimizer.step()                        # Update parameters

        epoch_loss += loss.item()  # Accumulate loss

        # Generate predictions: handle multi-class vs binary
        if outputs.shape[1] > 1:
            _, preds = torch.max(outputs, dim=1)
        else:
            preds = (torch.sigmoid(outputs) > 0.5).float()

        # Compute overall Dice score for the batch
        dice = dice_coefficient_metric(preds, targets)
        dice_score += dice.item()

        # Compute per-class Dice scores
        class_dice_scores = []
        for class_id in range(4):
            pred_class = (preds == class_id).float()
            target_class = (targets == class_id).float()
            intersection = (pred_class * target_class).sum()
            class_dice = (2. * intersection + 1e-6) / (pred_class.sum() + target_class.sum() + 1e-6)
            dice_per_class[class_id] += class_dice.item()
            class_dice_scores.append(class_dice.item())

        batch_count += 1
        # Print progress every 5 batches
        if (i + 1) % 5 == 0:
            print(f"Batch {i+1}/{len(dataloader)}, Loss: {loss.item():.4f}, Dice: {dice:.4f}")
            print(f"  Class Dice: Background: {class_dice_scores[0]:.4f}, \
                  NCR: {class_dice_scores[1]:.4f}, ED: {class_dice_scores[2]:.4f}, ET: {class_dice_scores[3]:.4f}")

        # Free memory
        del inputs, targets, outputs, preds
        torch.cuda.empty_cache()

    # Compute average metrics over all batches
    avg_loss = epoch_loss / batch_count
    avg_dice = dice_score / batch_count
    avg_dice_per_class = dice_per_class / batch_count

    return avg_loss, avg_dice, avg_dice_per_class


def validate_one_epoch(model, dataloader, criterion, device, scaler=None):
    """
    Validate the model for one epoch without gradient updates.
    Args:
        model: PyTorch model to evaluate.
        dataloader: DataLoader providing (inputs, targets).
        criterion: Loss function to compute validation loss.
        device: Torch device (cpu or cuda).
        scaler: Optional GradScaler for mixed precision during validation.
    Returns:
        avg_loss: Average validation loss.
        avg_dice: Average Dice score.
        avg_dice_per_class: Tensor of average Dice per class.
    """
    model.eval()  # Set model to evaluation mode
    epoch_loss = 0.0
    dice_score = 0.0
    dice_per_class = torch.zeros(4, device=device)
    batch_count = 0

    with torch.no_grad():  # Disable gradient calculation
        for i, (inputs, targets) in enumerate(dataloader):
            inputs = inputs.to(device)
            targets = targets.to(device)

            # Determine number of classes from criterion if available
            if hasattr(criterion, 'dice_loss') and hasattr(criterion.dice_loss, 'num_classes'):
                num_classes = criterion.dice_loss.num_classes
            else:
                num_classes = 4  # Fallback to 4 classes

            # Forward pass under mixed precision if scaler provided
            with torch.cuda.amp.autocast(enabled=(scaler is not None)):
                outputs = model(inputs)             # Logits from model
                loss = criterion(outputs, targets)  # Loss computation

            epoch_loss += loss.item()

            # Get predictions for metric computation
            if outputs.shape[1] > 1:
                _, preds = torch.max(outputs, dim=1)
            else:
                preds = (torch.sigmoid(outputs) > 0.5).float()

            # Compute overall Dice score
            dice = dice_coefficient_metric(preds, targets, num_classes=num_classes)
            dice_score += dice.item()

            # Accumulate per-class Dice scores
            for class_id in range(num_classes):
                pred_class = (preds == class_id).float()
                target_class = (targets == class_id).float()
                intersection = (pred_class * target_class).sum()
                class_dice = (2. * intersection + 1e-6) / (pred_class.sum() + target_class.sum() + 1e-6)
                dice_per_class[class_id] += class_dice.item()

            batch_count += 1
            # Free memory
            del inputs, targets, outputs, preds
            torch.cuda.empty_cache()

    # Average validation metrics
    avg_loss = epoch_loss / batch_count
    avg_dice = dice_score / batch_count
    avg_dice_per_class = dice_per_class / batch_count

    return avg_loss, avg_dice, avg_dice_per_class


class EarlyStopping:
    """
    Early stops the training if validation loss doesn't improve after a given patience.
    """
    def __init__(self, patience=7, min_delta=0.001):
        self.patience = patience      # Number of epochs to wait after last improvement
        self.min_delta = min_delta    # Minimum change to qualify as improvement
        self.counter = 0              # Counter for epochs without improvement
        self.best_score = None        # Best observed validation loss
        self.early_stop = False       # Flag to indicate whether to stop training

    def __call__(self, val_loss):
        # Initialize best_score at first call
        if self.best_score is None:
            self.best_score = val_loss
        # If no significant improvement, increment counter
        elif val_loss > self.best_score + self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True  # Trigger early stop
        else:
            # Improvement observed: reset counter and update best_score
            self.best_score = val_loss
            self.counter = 0

In [12]:
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HF_TOKEN")
api = HfApi()
repo_name = "dugoalberto/3D_U_Net"
checkpoint_dir = "./checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)

## Training (from new model)

In [ ]:
# Training loop with early stopping
num_epochs = 300  # Increased epochs with early stopping
best_val_dice = 0.0
early_stopping = EarlyStopping(patience=10)

# Lists to store metrics
train_losses = []
val_losses = []
train_dice_scores = []
val_dice_scores = []
learning_rates = []

for epoch in range(num_epochs):
    print(f"Epoch [{epoch+1}/{num_epochs}]")

    # Train
    train_loss, train_dice = train_one_epoch(model, train_loader, optimizer, criterion, device, scaler)
    train_losses.append(train_loss)
    train_dice_scores.append(train_dice)

    # Update learning rate
    current_lr = optimizer.param_groups[0]['lr']
    learning_rates.append(current_lr)
    if scheduler is not None and isinstance(scheduler, torch.optim.lr_scheduler.ReduceLROnPlateau):
            scheduler.step(val_loss)
    else: 
        scheduler.step()

    # Free memory before validation
    torch.cuda.empty_cache()

    # Validate
    val_loss, val_dice = validate_one_epoch(model, val_loader, criterion, device)
    val_losses.append(val_loss)
    val_dice_scores.append(val_dice)

    # Print epoch results
    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Train Dice: {train_dice:.4f}")
    print(f"Validation Loss: {val_loss:.4f}, Validation Dice: {val_dice:.4f}, LR: {current_lr:.6f}")

    if val_dice > best_val_dice:
        best_val_dice = val_dice
        best_model_path = os.path.join(checkpoint_dir, 'BratsCV_best.pt')
        torch.save(model, best_model_path)
        print(f"Best model saved with Dice score: {best_val_dice:.4f}!")

        # Upload best model to Hugging Face
        api.upload_file(
            path_or_fileobj=best_model_path,
            path_in_repo="BratsCV_best.pt",
            repo_id=repo_name,
            token=hf_token
        )
        print(f"Best model uploaded to Hugging Face!")

    # Save regular checkpoint every 5 epochs or final epoch
    if (epoch + 1) % 5 == 0 or epoch == num_epochs - 1:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        checkpoint_path = os.path.join(checkpoint_dir, f'BratsCV_epoch_{epoch+1}_{timestamp}.pt')

        # Save model state dictionary and optimizer state
        checkpoint = {
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict() if scheduler else None,
            'train_loss': train_loss,
            'val_loss': val_loss,
            'train_dice': train_dice,
            'val_dice': val_dice,
            'best_val_dice': best_val_dice,
            'learning_rate': current_lr
        }
        torch.save(checkpoint, checkpoint_path)

        # Upload checkpoint to Hugging Face
        try:
            api.upload_file(
                path_or_fileobj=checkpoint_path,
                path_in_repo=f"checkpoints/BratsCV_epoch_{epoch+1}.pt",
                repo_id=repo_name,
                token=hf_token
            )
            print(f"Checkpoint for epoch {epoch+1} uploaded to Hugging Face!")
        except Exception as e:
            print(f"Error uploading checkpoint to Hugging Face: {e}")

    early_stopping(val_loss)
    if early_stopping.early_stop:
        print(f"Early stopping triggered after {epoch+1} epochs!")
        final_model_path = os.path.join(checkpoint_dir, 'BratsCV_final.pt')
        torch.save(model, final_model_path)
        api.upload_file(
            path_or_fileobj=final_model_path,
            path_in_repo="BratsCV_final.pt",
            repo_id=repo_name,
            token=hf_token
        )
        print("Final model uploaded to Hugging Face!")
        break

    # Free memory after each epoch
    torch.cuda.empty_cache()

# Plot training and validation metrics
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Validation Loss')
plt.title('Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 3, 2)
plt.plot(train_dice_scores, label='Train Dice')
plt.plot(val_dice_scores, label='Validation Dice')
plt.title('Dice Score')
plt.xlabel('Epoch')
plt.ylabel('Dice')
plt.legend()

plt.subplot(1, 3, 3)
plt.plot(learning_rates)
plt.title('Learning Rate')
plt.xlabel('Epoch')
plt.ylabel('LR')

plt.tight_layout()
plt.savefig('training_metrics.png')
plt.show()

print(f"Training completed! Best validation Dice score: {best_val_dice:.4f}")

## Training (from restored model)

In [22]:
# Nome del checkpoint da recuperare
checkpoint_filename = "checkpoints/BratsCV_epoch_120.pt"
repo_name = "dugoalberto/3D_U_Net"
local_checkpoint_path = api.hf_hub_download(
        repo_id=repo_name,
        filename=checkpoint_filename,
        token=hf_token
)
checkpoint = torch.load(local_checkpoint_path)

# Verifica i parametri recuperati
print("Checkpoint Recovery Information:")
print("-" * 40)

# Stampa informazioni sull'epoch
print(f"Epoch: {checkpoint.get('epoch', 'Not found')}")

# Stampa metriche
print(f"Training Loss: {checkpoint.get('train_loss', 'Not found')}")
print(f"Validation Loss: {checkpoint.get('val_loss', 'Not found')}")
print(f"Training Dice Score: {checkpoint.get('train_dice', 'Not found')}")
print(f"Validation Dice Score: {checkpoint.get('val_dice', 'Not found')}")
print(f"Best Validation Dice Score: {checkpoint.get('best_val_dice', 'Not found')}")

model = ImprovedUNet3D()
model = torch.nn.DataParallel(model)
model.load_state_dict(checkpoint['model_state_dict'])
model.to(device)

Checkpoint Recovery Information:
----------------------------------------
Epoch: 120
Training Loss: 0.19613416381433707
Validation Loss: 0.2129683260982101
Training Dice Score: 0.8485563746114977
Validation Dice Score: 0.787887025121096
Best Validation Dice Score: 0.7913068532943726


/tmp/ipykernel_31/3430020675.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(local_checkpoint_path)


DataParallel(
  (module): ImprovedUNet3D(
    (enc1): Sequential(
      (0): Conv3d(4, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
      (1): InstanceNorm3d(16, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (2): LeakyReLU(negative_slope=0.01, inplace=True)
      (3): Conv3d(16, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
      (4): InstanceNorm3d(16, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (5): LeakyReLU(negative_slope=0.01, inplace=True)
    )
    (pool1): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (enc2): Sequential(
      (0): Conv3d(16, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
      (1): InstanceNorm3d(32, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (2): LeakyReLU(negative_slope=0.01, inplace=True)
      (3): Conv3d(32, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
      (4): Instan

In [23]:
optimizer = optim.AdamW(model.parameters(), lr=0.00004, weight_decay=5e-6)
#optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
#scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=30, eta_min=1e-6)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode='max',           # 'max' because we want to maximize validation accuracy
    factor=0.8,           # Multiply LR by this factor when reducing
    patience=2,           # Number of epochs with no improvement after which LR will be reduced
    verbose=True,         # Print message when LR is reduced
    threshold=0.1,      # Threshold for measuring improvement
    min_lr=1e-9           # Lower bound on the learning rate
)

num_epochs = 300  # Increased epochs with early stopping
start_epoch = checkpoint['epoch']
early_stopping = EarlyStopping(patience=10)
use_amp = True
scaler = torch.cuda.amp.GradScaler(enabled=use_amp)

/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
/tmp/ipykernel_31/603968276.py:18: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=use_amp)


In [ ]:
# Lists to store metrics
train_losses = []
val_losses = []
train_dice_scores = []
val_dice_scores = []
learning_rates = []

best_val_dice = checkpoint['best_val_dice']

# Restart the training
for epoch in range(start_epoch, num_epochs):
    print(f"Epoch [{epoch+1}/{num_epochs}]")
    # Train
    train_loss, train_dice = train_one_epoch(model, train_loader, optimizer, criterion, device, scaler)
    train_losses.append(train_loss)
    train_dice_scores.append(train_dice)

    # Free memory before validation
    torch.cuda.empty_cache()

    # Validate
    val_loss, val_dice = validate_one_epoch(model, val_loader, criterion, device)
    val_losses.append(val_loss)
    val_dice_scores.append(val_dice)
    
    # Update learning rate
    current_lr = optimizer.param_groups[0]['lr']
    print(current_lr)
    learning_rates.append(current_lr)
    if scheduler is not None and isinstance(scheduler, torch.optim.lr_scheduler.ReduceLROnPlateau):
            scheduler.step(val_loss)
    else :
        scheduler.step()
    #torch.cuda.empty_cache()
    # Print epoch results
    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Train Dice: {train_dice:.4f}")
    print(f"Validation Loss: {val_loss:.4f}, Validation Dice: {val_dice:.4f}")

    if val_dice > best_val_dice:
        best_val_dice = val_dice
        best_model_path = os.path.join(checkpoint_dir, 'BratsCV_best.pt')
        torch.save(model, best_model_path)
        print(f"Best model saved with Dice score: {best_val_dice:.4f}!")
        api.upload_file(
            path_or_fileobj=best_model_path,
            path_in_repo="BratsCV_best.pt",
            repo_id=repo_name,
            token=hf_token
        )
        print(f"Best model uploaded to Hugging Face!")
    if (epoch + 1) % 5 == 0 or epoch == num_epochs - 1:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        checkpoint_path = os.path.join(checkpoint_dir, f'BratsCV_epoch_{epoch+1}_{timestamp}.pt')

        # Save model state dictionary and optimizer state
        checkpoint = {
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict() if scheduler else None,
            'train_loss': train_loss,
            'val_loss': val_loss,
            'train_dice': train_dice,
            'val_dice': val_dice,
            'best_val_dice': best_val_dice,
            'learning_rate': current_lr
        }
        torch.save(checkpoint, checkpoint_path)

        # Upload checkpoint to Hugging Face
        try:
            api.upload_file(
                path_or_fileobj=checkpoint_path,
                path_in_repo=f"checkpoints/BratsCV_epoch_{epoch+1}.pt",
                repo_id=repo_name,
                token=hf_token
            )
            print(f"Checkpoint for epoch {epoch+1} uploaded to Hugging Face!")
        except Exception as e:
            print(f"Error uploading checkpoint to Hugging Face: {e}")

    # Check for early stopping
    early_stopping(val_loss)
    if early_stopping.early_stop:
        print(f"Early stopping triggered after {epoch+1} epochs!")

        # Save final model after early stopping
        final_model_path = os.path.join(checkpoint_dir, 'BratsCV_final.pt')
        torch.save(model, final_model_path)

        # Upload final model to Hugging Face
        api.upload_file(
            path_or_fileobj=final_model_path,
            path_in_repo="BratsCV_final.pt",
            repo_id=repo_name,
            token=hf_token
        )
        print("Final model uploaded to Hugging Face!")
        break